In [15]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict


In [16]:
# -------------- Model ---------------------
model = ChatOllama(
    model="qwen2.5:0.5b",
    temperature=0.7,
)

# -------------- State -----------------
class BlogState(TypedDict):
    topic: str
    title: str
    outline: str
    content: str
    refined_content: str



In [17]:
# ------------ Nodes ---------------------
def research_agent(state: BlogState) -> BlogState:
    """Research the topic and create an outline"""
    prompt = ChatPromptTemplate.from_template(
        "You are a research assistant. Create a detailed outline for a blog post about: {topic}\n\n"
        "Provide a structured outline with main points and subpoints."
    )
    chain = prompt | model
    result = chain.invoke({"topic": state["topic"]})
    state["outline"] = result.content
    return state

def title_agent(state: BlogState) -> BlogState:
    """Generate an engaging title"""
    prompt = ChatPromptTemplate.from_template(
        "Based on this topic: {topic}\n\n"
        "Create a catchy, SEO-friendly blog post title. Return ONLY the title, nothing else."
    )
    chain = prompt | model
    result = chain.invoke({"topic": state["topic"]})
    state["title"] = result.content.strip()
    return state

def writer_agent(state: BlogState) -> BlogState:
    """Write the blog content based on outline"""
    prompt = ChatPromptTemplate.from_template(
        "You are a professional blog writer. Write a comprehensive blog post based on:\n\n"
        "Topic: {topic}\n"
        "Title: {title}\n"
        "Outline: {outline}\n\n"
        "Write a well-structured, engaging blog post with introduction, body, and conclusion. "
        "Make it informative and easy to read. Minimum 500 words."
    )
    chain = prompt | model
    result = chain.invoke({
        "topic": state["topic"],
        "title": state["title"],
        "outline": state["outline"]
    })
    state["content"] = result.content
    return state

def editor_agent(state: BlogState) -> BlogState:
    """Review and refine the content"""
    prompt = ChatPromptTemplate.from_template(
        "You are an editor. Review and improve this blog post:\n\n"
        "Title: {title}\n\n"
        "{content}\n\n"
        "Improve clarity, fix any errors, and enhance readability. Return the refined version."
    )
    chain = prompt | model
    result = chain.invoke({
        "title": state["title"],
        "content": state["content"]
    })
    state["refined_content"] = result.content
    return state



In [18]:
# ----------------- Graph ---------------
graph = StateGraph(BlogState)

# Add nodes
graph.add_node("blog_research", research_agent)
graph.add_node("blog_title", title_agent)
graph.add_node("blog_writer", writer_agent)
graph.add_node("blog_editor", editor_agent)

# Define edges
graph.set_entry_point("blog_research")
graph.add_edge("blog_research", "blog_title")
graph.add_edge("blog_title", "blog_writer")
graph.add_edge("blog_writer", "blog_editor")
graph.add_edge("blog_editor", END)

# Compile the workflow
workflow = graph.compile()



In [20]:
# ------------------ Run ---------------
initial_state = {
    "topic": "Today's Love",
    "title": "",
    "outline": "",
    "content": "",
    "refined_content": ""
}

print("\nStarting blog generation...")
    
# Stream the execution to see progress
for step in workflow.stream(initial_state):
    node_name = list(step.keys())[0]
    print(f"Completed: {node_name}")

# Get final result
result = workflow.invoke(initial_state)
    
# Print results
print("\n" + "=" * 80)
print("BLOG POST GENERATION COMPLETE")
print("=" * 80)
print(f"\nTopic: {result['topic']}")
print(f"\nTitle: {result['title']}")
print(f"\n{'=' * 80}")
print("OUTLINE:")
print(f"{'=' * 80}")
print(result['outline'])
print(f"\n{'=' * 80}")
print("REFINED CONTENT:")
print(f"{'=' * 80}")
print(result['refined_content'])


Starting blog generation...
Completed: blog_research
titlerch

Completed: blog_writer
editor

Completed: __end__

BLOG POST GENERATION COMPLETE

Topic: Today's Love

Title: Love Meets Hate: A Social Media Analysis of Current Trends and Implications

OUTLINE:
Certainly! Here is a detailed outline for a blog post titled "Today's Love":

---

**Title:** "Today's Love: A Journey Through Contemporary Relationships"

**Introduction:**
- Brief introduction to the topic of today’s love
- Emphasize that this post will explore how contemporary relationships have changed, evolved, and become more complex over time.

**Body:**

1. **Historical Context:**
   - Explain how love has been a central theme in literature, art, and music throughout history.
   - Highlight key figures like Shakespeare, Mozart, and Beethoven who were deeply involved with their romantic partners.

2. **Modern Relationships:**
   - Discuss the evolution of relationships from traditional marriages to single life experiences.
